In [30]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import cv2
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ==================== 预处理核心函数 ====================
def clahe_enhance(image_np, clip_limit=2.0, grid_size=(8, 8)):
    """CLAHE对比度增强（适配RGB/灰度图像）"""
    if len(image_np.shape) == 2:
        gray = image_np
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)
        enhanced_gray = clahe.apply(gray)
        return enhanced_gray
    elif len(image_np.shape) == 3 and image_np.shape[2] == 3:
        gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)
        enhanced_gray = clahe.apply(gray)
        enhanced_rgb = cv2.cvtColor(enhanced_gray, cv2.COLOR_GRAY2RGB)
        return enhanced_rgb
    else:
        return image_np

def gamma_correction(image_np, gamma=1.2):
    """Gamma校正（提升/降低图像亮度）"""
    image_normalized = image_np / 255.0
    corrected = np.power(image_normalized, gamma)
    corrected = (corrected * 255).astype(np.uint8)
    return corrected

def show_preprocess_example(image_path, save_path="preprocess_example.png"):
    """展示预处理前后的对比效果"""
    raw_image = Image.open(image_path).convert('RGB')
    raw_image = raw_image.resize((512, 512), Image.BILINEAR)
    raw_np = np.array(raw_image)
    
    clahe_np = clahe_enhance(raw_np)
    gamma_np = gamma_correction(clahe_np, gamma=1.2)
    
    print("\n" + "="*80)
    print("📊 预处理前后像素值统计（示例）")
    print("="*80)
    print(f"原始图像 - 均值: {np.mean(raw_np):.2f}, 标准差: {np.std(raw_np):.2f}, 最小值: {np.min(raw_np)}, 最大值: {np.max(raw_np)}")
    print(f"CLAHE增强后 - 均值: {np.mean(clahe_np):.2f}, 标准差: {np.std(clahe_np):.2f}, 最小值: {np.min(clahe_np)}, 最大值: {np.max(clahe_np)}")
    print(f"Gamma校正后 - 均值: {np.mean(gamma_np):.2f}, 标准差: {np.std(gamma_np):.2f}, 最小值: {np.min(gamma_np)}, 最大值: {np.max(gamma_np)}")
    print("="*80)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    axes[0].imshow(raw_np)
    axes[0].set_title("原始图像", fontsize=14)
    axes[0].axis('off')
    
    axes[1].imshow(clahe_np)
    axes[1].set_title("CLAHE对比度增强", fontsize=14)
    axes[1].axis('off')
    
    axes[2].imshow(gamma_np)
    axes[2].set_title("Gamma校正 (γ=1.2)", fontsize=14)
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"\n✅ 预处理对比图已保存至: {save_path}")
    print("="*80 + "\n")

# ==================== 数据加载和预处理 ====================
class RetinaVesselDataset(Dataset):
    def __init__(self, image_dir, label_dir=None, transform=None, is_train=True, image_size=512):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.is_train = is_train
        self.image_size = image_size

        self.image_files = []
        for file in sorted(os.listdir(image_dir)):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.gif')):
                self.image_files.append(file)

        print(f"找到 {len(self.image_files)} 张图像")
        
        if self.image_files and self.is_train:
            first_image_path = os.path.join(self.image_dir, self.image_files[0])
            show_preprocess_example(first_image_path)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)

        image = Image.open(img_path).convert('RGB')

        if self.is_train and self.label_dir:
            label_name = img_name
            label_path = os.path.join(self.label_dir, label_name)

            if os.path.exists(label_path):
                label = Image.open(label_path).convert('L')
            else:
                label = Image.new('L', image.size, 0)

            image = image.resize((self.image_size, self.image_size), Image.BILINEAR)
            label = label.resize((self.image_size, self.image_size), Image.NEAREST)

            image_np = np.array(image)
            label_np = np.array(label)

            # 预处理
            image_np = clahe_enhance(image_np)
            image_np = gamma_correction(image_np, 1.2)
            
            label_np = (label_np > 128).astype(np.float32)

            image_tensor = transforms.ToTensor()(image_np)
            label_tensor = torch.from_numpy(label_np).unsqueeze(0)

            image_tensor = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                std=[0.229, 0.224, 0.225])(image_tensor)

            return image_tensor, label_tensor, img_name
        else:
            image = image.resize((self.image_size, self.image_size), Image.BILINEAR)
            image_np = np.array(image)
            
            image_np = clahe_enhance(image_np)
            image_np = gamma_correction(image_np, 1.2)
            
            image_tensor = transforms.ToTensor()(image_np)
            image_tensor = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                std=[0.229, 0.224, 0.225])(image_tensor)

            return image_tensor, img_name

    def get_image_id(self, idx):
        img_name = self.image_files[idx]
        return os.path.splitext(img_name)[0]

# ==================== UNet模型定义（添加Dropout） ====================
class DoubleConv(nn.Module):
    """(卷积 => BN => ReLU => Dropout) * 2"""
    def __init__(self, in_channels, out_channels, dropout_rate=0.2):  # 新增dropout_rate参数
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(dropout_rate),  # 2D Dropout，针对特征图
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(dropout_rate)   # 第二层卷积后也加Dropout
        )

    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    """下采样：最大池化 + DoubleConv（带Dropout）"""
    def __init__(self, in_channels, out_channels, dropout_rate=0.2):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels, dropout_rate)  # 传递dropout参数
        )

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    """上采样 + 跳跃连接（带Dropout）"""
    def __init__(self, in_channels, out_channels, bilinear=True, dropout_rate=0.2):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, dropout_rate)  # 传递dropout参数
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels, dropout_rate)  # 传递dropout参数

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1, bilinear=True, dropout_rate=0.2):  # 新增dropout_rate参数
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        # 编码器部分（下采样）：逐层设置Dropout（浅层小，深层稍大）
        self.inc = DoubleConv(n_channels, 64, dropout_rate=0.1)  # 输入层Dropout稍小（0.1）
        self.down1 = Down(64, 128, dropout_rate=0.15)            # 第一层下采样（0.15）
        self.down2 = Down(128, 256, dropout_rate=0.2)            # 第二层下采样（0.2）
        self.down3 = Down(256, 512, dropout_rate=0.2)            # 第三层下采样（0.2）
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor, dropout_rate=0.25)# 最深层Dropout稍大（0.25）

        # 解码器部分（上采样）
        self.up1 = Up(1024, 512 // factor, bilinear, dropout_rate=0.25)
        self.up2 = Up(512, 256 // factor, bilinear, dropout_rate=0.2)
        self.up3 = Up(256, 128 // factor, bilinear, dropout_rate=0.15)
        self.up4 = Up(128, 64, bilinear, dropout_rate=0.1)

        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)  # 输出层不加Dropout

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return torch.sigmoid(logits)

# ==================== 损失函数和评价指标 ====================
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, predictions, targets):
        predictions = predictions.view(-1)
        targets = targets.view(-1)
        intersection = (predictions * targets).sum()
        dice = (2. * intersection + self.smooth) / (predictions.sum() + targets.sum() + self.smooth)
        return 1 - dice

def calculate_dice_coefficient(predictions, targets, smooth=1e-6):
    preds_binary = (predictions > 0.5).float()
    preds_flat = preds_binary.view(-1)
    targets_flat = targets.view(-1)
    intersection = (preds_flat * targets_flat).sum()
    union = preds_flat.sum() + targets_flat.sum()
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice.item()

# ==================== 训练函数 ====================
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0.0
    epoch_dice = 0.0
    total_batches = len(train_loader)

    for batch_idx, (images, masks, _) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        dice = calculate_dice_coefficient(outputs, masks)
        
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_dice += dice

    avg_loss = epoch_loss / total_batches
    avg_dice = epoch_dice / total_batches
    return avg_loss, avg_dice

def validate_epoch(model, val_loader, criterion, device):
    model.eval()
    epoch_loss = 0.0
    epoch_dice = 0.0
    total_batches = len(val_loader)

    with torch.no_grad():
        for batch_idx, (images, masks, _) in enumerate(tqdm(val_loader, desc="Validation", leave=False)):
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            loss = criterion(outputs, masks)
            dice = calculate_dice_coefficient(outputs, masks)

            epoch_loss += loss.item()
            epoch_dice += dice

    avg_loss = epoch_loss / total_batches
    avg_dice = epoch_dice / total_batches
    return avg_loss, avg_dice

def train_model(model, train_loader, val_loader, num_epochs=50, learning_rate=1e-4):
    print("开始训练模型（带Dropout正则化）...")
    print("保存策略：基于验证集Dice系数（越高越好）")

    criterion = DiceLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5, factor=0.5)

    train_loss_history = []
    val_loss_history = []
    train_dice_history = []
    val_dice_history = []
    
    best_val_dice = 0.0
    best_epoch = 0
    best_model_state = None

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        print("-" * 50)

        train_loss, train_dice = train_epoch(model, train_loader, criterion, optimizer, device)
        train_loss_history.append(train_loss)
        train_dice_history.append(train_dice)

        val_loss, val_dice = validate_epoch(model, val_loader, criterion, device)
        val_loss_history.append(val_loss)
        val_dice_history.append(val_dice)

        scheduler.step(val_dice)

        print(f"Train Loss: {train_loss:.4f} | Train Dice: {train_dice:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | Val   Dice: {val_dice:.4f} (Best: {best_val_dice:.4f})")
        print(f"Current LR: {optimizer.param_groups[0]['lr']:.6f}")

        if val_dice > best_val_dice:
            best_val_dice = val_dice
            best_epoch = epoch + 1
            best_model_state = model.state_dict().copy()
            
            torch.save({
                'epoch': best_epoch,
                'model_state_dict': best_model_state,
                'best_val_dice': best_val_dice,
            }, 'best_model_by_dice.pth')
            
            print(f"✅ 更新最佳模型 | Epoch: {best_epoch} | Best Val Dice: {best_val_dice:.4f}")
        else:
            print(f"❌ 未更新最佳模型 | 当前Val Dice: {val_dice:.4f} < 历史最佳: {best_val_dice:.4f}")

    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"\n=====================================")
        print(f"训练完成！加载最佳模型（基于验证集Dice）")
        print(f"最佳Epoch: {best_epoch}")
        print(f"最佳Val Dice: {best_val_dice:.4f}")
        print(f"=====================================")

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    ax1.plot(train_loss_history, label='Train Loss', marker='o', markersize=4)
    ax1.plot(val_loss_history, label='Val Loss', marker='s', markersize=4)
    if best_epoch > 0:
        ax1.scatter(best_epoch-1, val_loss_history[best_epoch-1], 
                   color='red', s=100, marker='*', label=f'Best Dice (Epoch {best_epoch})')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(train_dice_history, label='Train Dice', marker='o', markersize=4, color='green')
    ax2.plot(val_dice_history, label='Val Dice', marker='s', markersize=4, color='red')
    if best_epoch > 0:
        ax2.scatter(best_epoch-1, best_val_dice, 
                   color='red', s=100, marker='*', label=f'Best Dice ({best_val_dice:.4f})')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Dice Coefficient')
    ax2.set_title('Training and Validation Dice Coefficient')
    ax2.set_ylim(0, 1)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.close()

    history_df = pd.DataFrame({
        'epoch': range(1, num_epochs+1),
        'train_loss': train_loss_history,
        'val_loss': val_loss_history,
        'train_dice': train_dice_history,
        'val_dice': val_dice_history
    })
    history_df.to_csv('training_history.csv', index=False)
    print(f"\n训练历史已保存到: training_history.csv")

    return model

# ==================== 预测函数 ====================
def predict_test_set(model, test_loader, threshold=0.5):
    model.eval()
    predictions = {}

    with torch.no_grad():
        for images, img_names in tqdm(test_loader, desc="Predicting"):
            images = images.to(device)
            outputs = model(images)
            preds = (outputs > threshold).float()

            for i in range(len(img_names)):
                img_name = img_names[i]
                img_id = os.path.splitext(img_name)[0]
                pred_mask = preds[i, 0].cpu().numpy()
                pred_mask_resized = cv2.resize(pred_mask, (512, 512), interpolation=cv2.INTER_NEAREST)
                mask_uint8 = np.where(pred_mask_resized > 0.5, 0, 255).astype(np.uint8)
                predictions[img_id] = mask_uint8

    return predictions

# ==================== Run-length编码函数 ====================
def rle_encode(mask):
    pixels = mask.flatten()
    dots = np.where(pixels == 0)[0]

    if len(dots) == 0:
        return ""

    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev + 1):
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b

    return ' '.join([str(r) for r in run_lengths])

# ==================== 保存预测结果 ====================
def save_predictions_to_csv(predictions, output_csv='submission.csv'):
    df = pd.DataFrame(columns=['Id', 'Predicted'])
    ids = []
    rle_strings = []

    try:
        sorted_ids = sorted(predictions.keys(), key=lambda x: int(x))
    except:
        sorted_ids = sorted(predictions.keys())

    for img_id in sorted_ids:
        mask = predictions[img_id]
        rle_str = rle_encode(mask)
        ids.append(img_id)
        rle_strings.append(rle_str)

    df['Id'] = ids
    df['Predicted'] = rle_strings
    df.to_csv(output_csv, index=False)
    print(f"预测结果已保存到: {output_csv}")
    print(f"共保存了 {len(df)} 个预测结果")

# ==================== 主函数 ====================
def main():
    print("=" * 60)
    print("视网膜血管分割 - 训练与预测（集成CLAHE+Gamma+Dropout）")
    print("=" * 60)

    # 设置路径
    train_image_dir = '/kaggle/input/task-5-vesselsegmentation-2025/segmentation/train/image'
    train_label_dir = '/kaggle/input/task-5-vesselsegmentation-2025/segmentation/train/label'
    test_image_dir = '/kaggle/input/task-5-vesselsegmentation-2025/segmentation/test/image'
    output_csv = 'submission.csv'

    # 检查路径
    print("\n检查路径...")
    paths_to_check = [
        (train_image_dir, "训练图像"),
        (train_label_dir, "训练标签"),
        (test_image_dir, "测试图像")
    ]

    all_paths_exist = True
    for path, desc in paths_to_check:
        if os.path.exists(path):
            print(f"✓ {desc}路径: {path}")
        else:
            print(f"✗ {desc}路径不存在: {path}")
            all_paths_exist = False

    if not all_paths_exist:
        print("\n警告: 部分路径不存在，将使用备用路径...")
        train_image_dir = './train/image' if os.path.exists('./train/image') else train_image_dir
        train_label_dir = './train/label' if os.path.exists('./train/label') else train_label_dir
        test_image_dir = './test/image' if os.path.exists('./test/image') else test_image_dir

    # 准备数据
    print("\n" + "=" * 60)
    print("第1步：准备数据（集成CLAHE+Gamma预处理）")
    print("=" * 60)

    train_dataset = RetinaVesselDataset(
        image_dir=train_image_dir,
        label_dir=train_label_dir,
        is_train=True,
        image_size=512
    )

    print(f"训练集大小: {len(train_dataset)}")

    val_size = min(50, int(len(train_dataset) * 0.2))
    train_size = len(train_dataset) - val_size

    train_dataset, val_dataset = random_split(
        train_dataset,
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )

    print(f"训练集: {len(train_dataset)}, 验证集: {len(val_dataset)}")

    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)

    # 训练模型（初始化带Dropout的UNet）
    print("\n" + "=" * 60)
    print("第2步：训练模型（带Dropout正则化）")
    print("=" * 60)

    # 初始化模型时指定dropout_rate（也可使用默认值）
    model = UNet(n_channels=3, n_classes=1, bilinear=True, dropout_rate=0.2)
    model = model.to(device)
    print(f"模型参数量: {sum(p.numel() for p in model.parameters()):,}")
    
    train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=120,
        learning_rate=1e-3
    )

    # 预测
    print("\n" + "=" * 60)
    print("第3步：准备测试数据")
    print("=" * 60)

    test_dataset = RetinaVesselDataset(
        image_dir=test_image_dir,
        is_train=False,
        image_size=512
    )

    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)
    print(f"测试集大小: {len(test_dataset)}")

    print("\n" + "=" * 60)
    print("第4步：进行预测")
    print("=" * 60)

    predictions = predict_test_set(model, test_loader, threshold=0.5)

    print("\n" + "=" * 60)
    print("第5步：保存结果")
    print("=" * 60)

    save_predictions_to_csv(predictions, output_csv)

    print("\n" + "=" * 60)
    print("完成！")
    print("=" * 60)

if __name__ == "__main__":
    print("开始执行血管分割训练与预测脚本（集成CLAHE+Gamma+Dropout）...")

    torch.manual_seed(42)
    np.random.seed(42)

    try:
        main()
    except Exception as e:
        print(f"\n错误: {e}")
        print("=" * 60)
        print("请检查路径和环境配置！")
        print("=" * 60)

Using device: cuda
开始执行血管分割训练与预测脚本（集成CLAHE+Gamma+Dropout）...
视网膜血管分割 - 训练与预测（集成CLAHE+Gamma+Dropout）

检查路径...
✓ 训练图像路径: /kaggle/input/task-5-vesselsegmentation-2025/segmentation/train/image
✓ 训练标签路径: /kaggle/input/task-5-vesselsegmentation-2025/segmentation/train/label
✓ 测试图像路径: /kaggle/input/task-5-vesselsegmentation-2025/segmentation/test/image

第1步：准备数据（集成CLAHE+Gamma预处理）
找到 20 张图像

📊 预处理前后像素值统计（示例）
原始图像 - 均值: 73.94, 标准差: 57.73, 最小值: 0, 最大值: 255
CLAHE增强后 - 均值: 86.60, 标准差: 53.79, 最小值: 1, 最大值: 246
Gamma校正后 - 均值: 73.01, 标准差: 50.11, 最小值: 0, 最大值: 244

✅ 预处理对比图已保存至: preprocess_example.png

训练集大小: 20
训练集: 16, 验证集: 4

第2步：训练模型（带Dropout正则化）
模型参数量: 13,395,329
开始训练模型（带Dropout正则化）...
保存策略：基于验证集Dice系数（越高越好）

Epoch 1/120
--------------------------------------------------


Train Loss: 0.7579 | Train Dice: 0.3156
Val   Loss: 0.8286 | Val   Dice: 0.0012 (Best: 0.0000)
Current LR: 0.001000
✅ 更新最佳模型 | Epoch: 1 | Best Val Dice: 0.0012

Epoch 2/120
--------------------------------------------------


Train Loss: 0.6916 | Train Dice: 0.4805
Val   Loss: 0.7317 | Val   Dice: 0.5319 (Best: 0.0012)
Current LR: 0.001000
✅ 更新最佳模型 | Epoch: 2 | Best Val Dice: 0.5319

Epoch 3/120
--------------------------------------------------


Train Loss: 0.6604 | Train Dice: 0.5523
Val   Loss: 0.6420 | Val   Dice: 0.6307 (Best: 0.5319)
Current LR: 0.001000
✅ 更新最佳模型 | Epoch: 3 | Best Val Dice: 0.6307

Epoch 4/120
--------------------------------------------------


Train Loss: 0.6309 | Train Dice: 0.6091
Val   Loss: 0.5992 | Val   Dice: 0.7053 (Best: 0.6307)
Current LR: 0.001000
✅ 更新最佳模型 | Epoch: 4 | Best Val Dice: 0.7053

Epoch 5/120
--------------------------------------------------


...

Train Loss: 0.2246 | Train Dice: 0.7847
Val   Loss: 0.2000 | Val   Dice: 0.8067 (Best: 0.8072)
Current LR: 0.000002
❌ 未更新最佳模型 | 当前Val Dice: 0.8067 < 历史最佳: 0.8072

Epoch 119/120
--------------------------------------------------


Train Loss: 0.2231 | Train Dice: 0.7857
Val   Loss: 0.2004 | Val   Dice: 0.8068 (Best: 0.8072)
Current LR: 0.000001
❌ 未更新最佳模型 | 当前Val Dice: 0.8068 < 历史最佳: 0.8072

Epoch 120/120
--------------------------------------------------


Train Loss: 0.2313 | Train Dice: 0.7772
Val   Loss: 0.2003 | Val   Dice: 0.8066 (Best: 0.8072)
Current LR: 0.000001
❌ 未更新最佳模型 | 当前Val Dice: 0.8066 < 历史最佳: 0.8072

训练完成！加载最佳模型（基于验证集Dice）
最佳Epoch: 83
最佳Val Dice: 0.8072

训练历史已保存到: training_history.csv

第3步：准备测试数据
找到 20 张图像
测试集大小: 20

第4步：进行预测


Predicting: 100%|██████████| 5/5 [00:00<00:00,  5.94it/s]



第5步：保存结果
预测结果已保存到: submission.csv
共保存了 20 个预测结果

完成！
